In [1]:
import warnings
import pandas as pd

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

def item_group(df):
    #new column for Item group
    df['ITEM_GROUP'] = None
    
    
    # Loop through the rows and add the current_group to the ITEM_GROUP column
    current_group = None
    
    for idx, row in df.iterrows():
        #check if other columns are none
        if (pd.isna(row['FROM']) and pd.isna(row['TO']) and pd.isna(row['ENDS']) and 
            pd.isna(row['DESCRIPTION']) and pd.isna(row['STANDARD']) and pd.isna(row['MATERIAL'])):
            current_group = row['ITEM'] #current group
        df.at[idx, 'ITEM_GROUP'] = current_group #update the current_group
    df = df.dropna()
    return df

In [2]:
import warnings
import pandas as pd

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

import re

# Mapping for common fraction characters
fraction_mapping = {
    "½": 0.5,
    "1½": 1.5,
    "1\n½":1.5,
    "¾": 0.75,
    "1 ½": 1.5,
}

# Function to convert mixed fractions and fractions to decimal
def convert_fraction_to_decimal(value):
    # Split the value based on spaces (for mixed fractions like "1 ½")
    parts = value.split()
    
    # If it's a mixed fraction (e.g., "1 ½")
    if len(parts) == 2:
        whole_number = int(parts[0])
        fraction = fraction_mapping.get(parts[1], 0)
        return whole_number + fraction
    
    # If it's only a fraction (e.g., "½")
    if len(parts) == 1:
        return fraction_mapping.get(parts[0], value)  # Return as-is if not found

    return value  # If no fraction found, return the original value


In [66]:
import pdfplumber
import pandas as pd
import warnings
import pandas as pd

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

def extract_table(pdf):
    spec = pdf.replace(' Rev1.pdf','')
    fraction_lst = ["½","¾","1 ½","1½","1\n½"]
    
    print(pdf)
    
    # Path to your PDF file
    pdf_path = pdf
    print(pdf_path)
    #rating pattern
    pattern = r"CL\.\d+"
    
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        mapping_table = []
        
        # Erroneoud Tables
        er_dfs = []
        
        # Iterate through all the pages in the PDF
        for page_number, page in enumerate(pdf.pages, start=1):
            # Extract tables from the page
            tables = page.extract_tables()

            # If tables are found, process them
            for table in tables:
                # Convert the table into a DataFrame for easier manipulation
                df = pd.DataFrame(table)
                # Append the extracted table to the list
                if df.shape[1] > 5 and df.shape[1] < 15 and page_number >1:
#                     if pdf == ''
                    df.loc[df[1].isin(fraction_lst),1] = df.loc[df[1].isin(fraction_lst),1].apply(convert_fraction_to_decimal)
                    df.loc[df[2].isin(fraction_lst),2] = df.loc[df[1].isin(fraction_lst),2].apply(convert_fraction_to_decimal)
                    df.columns = df.iloc[0]  # Set row 0 as header
                    df = df.drop(0)
                    try:
                        df = df.drop(1)
                    except:
                        df = df.copy()
                    try:
                        df = df.drop('NOTES',axis=1)
                    except:
                        df = df.copy()
                    try:
                        df.columns = ['ITEM','FROM','TO','ENDS','DESCRIPTION','STANDARD','MATERIAL']
                        all_tables.append((df))
                    except:
                        df['PDF'] = spec
                        df['Page_Number'] = page_number
                        er_dfs.append(df)
                        
                elif df.shape[1] > 4:
                    df = df.T
                    df.loc[df[0].isin(fraction_lst),0] = df.loc[df[0].isin(fraction_lst),0].apply(convert_fraction_to_decimal)
                    df.columns = df.iloc[0]
                    df.drop(0,inplace=True)
                    mapping_table.append((df))
        #all table combined            
        df_ = pd.concat(all_tables,ignore_index=True)
        df_ = item_group(df_)
        df_['FROM'] = round(df_['FROM'].astype(float),1)
        df_['TO'] = round(df_['TO'].astype(float),1)
        print(mapping_table)
        #get the group
        try:
            df_specs = pd.concat([mapping_table[1],mapping_table[2]],ignore_index=True)
        except:
            try:
                df_specs = mapping_table[1]
            except:
                df_specs = mapping_table[0]
        print(df_specs.columns)
        try:
            df_specs['NPS'] = round(df_specs['NPS'].astype(float),1)
        except:
            df_specs['DN'] = round(df_specs['DN'].astype(float),1)
        result_df = pd.DataFrame()
        # Iterate over each row in df1
        for index, row in df_.iterrows():
            try:
            # Filter sizes from df2 that fall within the range [FROM, TO]
                filtered_sizes = df_specs[(df_specs['NPS'] >= row['FROM']) & (df_specs['NPS'] <= row['TO'])]
            except:
                filtered_sizes = df_specs[(df_specs['DN'] >= row['FROM']) & (df_specs['DN'] <= row['TO'])]


            matches = re.findall(pattern, row['DESCRIPTION'])
            # For each matching size, create a new row in result_df
            for idx, rw in filtered_sizes.iterrows():

                if len(matches) == 0:
                    rating = '-'
                else:
                    rating = matches[0]

                # Determine the SCH1 value
                if rw['SCHEDULE'] == '-':
                    sch1 = rw['THICKNESS']
                else:
                    sch1 = rw['SCHEDULE']
                try:
                    nips = rw['NPS']
                except:
                    nips = rw['DN']
                result_df = result_df.append({
                    'ITEM_GROUP': row['ITEM_GROUP'],
                    'ITEM': row['ITEM'],
                    'FROM': row['FROM'],
                    'TO': row['TO'],
                    'SIZE1 (NPS)': nips,
                    'SIZE2 (NPS)': "",
                    'DESCRIPTION': row['DESCRIPTION'],
                    'MATERIAL': row['MATERIAL'],
                    'ENDS': row['ENDS'],
                    'STANDARD': row['STANDARD'],          
                    'RATING': rating,
                    'SCH1': sch1

                }, ignore_index=True)
    result_df['PIPING SPEC'] = spec
    return result_df,er_dfs

In [67]:
# #Step1
# pdf_um = pd.read_excel('List_of_PDFs_Unmapped.xlsx')['Un Mapped pdfs'].unique()
# err_pdf = []
# rem_pdf = []

# # The following list contains list of erroneous tables
# erroneous_tables = []

# for pdf in pdf_um:
#     try:
#         rem_pdf.append(extract_table(pdf)[0])
#         erroneous_tables.append(extract_table(pdf)[1])
#     except:
#         err_pdf.append(pdf)


# Step 1: Read the Excel file to get the list of PDF names
pdf_um = pd.read_excel('List_of_PDFs_Unmapped.xlsx')['Un Mapped pdfs'].unique()
err_pdf = []
rem_pdf = []

# The following list contains list of erroneous tables
erroneous_tables = []

# Only process the specific PDF file 'AC1A3B-FA'
pdf_to_process = "AC1A3B-FA Rev1.pdf"

# Check if the specific PDF is in the list of 'Un Mapped pdfs'
if pdf_to_process in pdf_um:
    try:
        rem_pdf.append(extract_table(pdf_to_process)[0])  # Extract and store the main table
        erroneous_tables.append(extract_table(pdf_to_process)[1])  # Store any erroneous tables
    except:
        err_pdf.append(pdf_to_process)  # Add to err_pdf if extraction fails

AC1A3B-FA Rev1.pdf
AC1A3B-FA Rev1.pdf
[0 TEMP. PRESS.
1   -29   19.6
2     0   19.6
3    38   19.6
4    50   19.2
5   100   17.7
6   150   15.8, 0    NPS SCHEDULE THICKNESS
1    0.5      XXS      7.47
2   0.75      160      5.56
3      1      160      6.35
4    1.5      160      7.14
5      2      160      8.74
6      3       80      7.62
7      4       80      8.56
8      6       40      7.11
9      8       30      7.04
10    10       30      7.80
11    12       30      8.38, 0 NPS SCHEDULE THICKNESS
1  14       20      7.92
2  16       20      7.92
3  18       20      7.92
4  20       20      9.53
5  24       20      9.53]
Index(['NPS', 'SCHEDULE', 'THICKNESS'], dtype='object', name=0)


In [ ]:
result_df['PIPING SPEC']

In [ ]:
df_erl = pd.read_excel('Piping materail class index Appendix E1 Rev 3.xlsx')
df_now = pd.concat(rem_pdf,ignore_index=True)
pd.concat([df_erl,df_now], ignore_index=True).to_excel('Piping materail class index Appendix E1 Rev 4.xlsx',index=False)

In [ ]:
import pickle

In [ ]:
with open('erroneous_tables.pkl','wb') as f:
    pickle.dump(erroneous_tables,f)

In [ ]:
dir(pickle)

In [ ]:
import os
pdf_list = os.listdir()
pdf_list.remove(pdf_list[0])
pdf_list.remove(pdf_list[-1])

In [ ]:
for pdf_index in range (5, len(err_pdf)):
    pdf = err_pdf[pdf_index]
    extract_table(pdf)[0]

In [ ]:
len(df_list)+len(erroneous_pdf)

In [ ]:
df_final = pd.concat(df_list,ignore_index=True)

In [ ]:
df_final.to_excel('Piping materail class index Appendix E1 Rev 3.xlsx',index=False)

In [ ]:
pd.DataFrame({'Un Mapped pdfs':erroneous_pdf}).to_excel('List_of_PDFs_Unmapped.xlsx',index=False)

In [ ]:
err_pdf

In [ ]:
#Step1
pdf_um = pd.read_excel('List_of_PDFs_Unmapped.xlsx')['Un Mapped pdfs'].unique()
err_pdf = []
rem_pdf = []

# The following list contains list of erroneous tables
erroneous_tables = []

for pdf in pdf_um:
    try:
        rem_pdf.append(extract_table(pdf)[0])
        erroneous_tables.append(extract_table(pdf)[1])
    except:
        err_pdf.append(pdf)

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

...

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812\1272026280.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_10812

FC1A1A-FA-Rev1.pdf
FC1S3C-FA-Rev1.pdf
FC1S3C-JH-Rev1.pdf
FC1S3E-FA-Rev1.pdf
FC1S6C-FA-Rev1.pdf
HC2S1C-JH Rev1.pdf


In [ ]:
# Step2
rm_pdf = []
er_pdf = []
for pdf in err_pdf:
    try:
        rm_pdf.append(extract_table(pdf))
    except:
        er_pdf.append(pdf)

AP3A0F-FA Rev1.pdf
AP4A0K-FA Rev1.pdf
AS1S0A-FC Rev1.pdf
BL1H4D-FH Rev1.pdf
BP3S0C-FA Rev1.pdf
BP3S0E-FA Rev1.pdf
BS1S1A-FB Rev1.pdf
CC1A1B-FH Rev1.pdf
CC1A3B-FH Rev1.pdf
CC1P3B-FH Rev1.pdf
CC1S3E-FA Rev1.pdf
DC1S6A-FH Rev1.pdf
DC1S6C-FA Rev1.pdf
DS1A0A-FC Rev1.pdf
DS1S0A-FC Rev1.pdf
FC1A1A-FA-Rev1.pdf
FC1S3C-FA-Rev1.pdf
FC1S3C-JH-Rev1.pdf
FC1S3E-FA-Rev1.pdf
FC1S6C-FA-Rev1.pdf
HC2S1C-JH Rev1.pdf


In [ ]:
# Step3
rm3_pdf = []
er3_pdf = []
for pdf in 

In [ ]:
extract_table('AP2A0F-FA Rev1.pdf')

C:\Users\s.hassan\AppData\Local\Temp\ipykernel_6728\702625448.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_6728\702625448.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_6728\702625448.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_6728\702625448.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
C:\Users\s.hassan\AppData\Local\Temp\ipykernel_6728\7026

(                          ITEM_GROUP       ITEM  FROM     TO  ...         STANDARD  RATING   SCH1 PIPING SPEC
 0    PIPE (NOTE 143,190,191,192,194)       PIPE  63.0  900.0  ...         MFG STD.       -  CALC.   AP2A0F-FA
 1    PIPE (NOTE 143,190,191,192,194)       PIPE  63.0  900.0  ...         MFG STD.       -  CALC.   AP2A0F-FA
 2    PIPE (NOTE 143,190,191,192,194)       PIPE  63.0  900.0  ...         MFG STD.       -  CALC.   AP2A0F-FA
 3    PIPE (NOTE 143,190,191,192,194)       PIPE  63.0  900.0  ...         MFG STD.       -  CALC.   AP2A0F-FA
 4    PIPE (NOTE 143,190,191,192,194)       PIPE  63.0  900.0  ...         MFG STD.       -  CALC.   AP2A0F-FA
 ..                               ...        ...   ...    ...  ...              ...     ...    ...         ...
 242               VALVES (NOTE - 81)  BUTTERFLY  90.0  900.0  ...  API 609,\nCAT.A  CL.150  CALC.   AP2A0F-FA
 243               VALVES (NOTE - 81)  BUTTERFLY  90.0  900.0  ...  API 609,\nCAT.A  CL.150  CALC.   AP2A0F-FA
 

In [ ]:
import pdfplumber
import pandas as pd
import re
import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

def convert_fraction_to_decimal(fraction):
    """Helper function to convert fractions to decimal."""
    # Handle simple fraction-to-decimal conversion (e.g., 1/2 -> 0.5)
    fractions = {
        "½": 0.5,
        "¾": 0.75,
        "1 ½": 1.5,
        "1½": 1.5,
        "1\n½": 1.5
    }
    return fractions.get(fraction, fraction)

def extract_table(pdf):
    # Remove ' Rev1.pdf' from the file name for spec identification
    spec = pdf.replace(' Rev1.pdf', '')
    fraction_lst = ["½", "¾", "1 ½", "1½", "1\n½"]

    # Path to your PDF file
    pdf_path = pdf
    pattern = r"CL\.\d+"  # Rating pattern for matching

    # Open the PDF file using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []  # List to store all processed tables
        mapping_table = []  # List for the mapping tables (NPS/DN values)
        er_dfs = []  # List to store erroneous DataFrames

        # Iterate through all pages in the PDF
        for page_number, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables()  # Extract tables from the page

            for table in tables:
                # Convert the table into a DataFrame for easier manipulation
                df = pd.DataFrame(table)

                if df.shape[1] > 5 and df.shape[1] < 15 and page_number > 1:
                    df.loc[df[1].isin(fraction_lst), 1] = df.loc[df[1].isin(fraction_lst), 1].apply(convert_fraction_to_decimal)
                    df.loc[df[2].isin(fraction_lst), 2] = df.loc[df[1].isin(fraction_lst), 2].apply(convert_fraction_to_decimal)
                    df.columns = df.iloc[0]  # Set row 0 as header
                    df = df.drop(0)
                    try:
                        df = df.drop(1)
                    except:
                        df = df.copy()
                    try:
                        df = df.drop('NOTES', axis=1)
                    except:
                        df = df.copy()
                    try:
                        df.columns = ['ITEM', 'FROM', 'TO', 'ENDS', 'DESCRIPTION', 'STANDARD', 'MATERIAL']
                        all_tables.append(df)  # Append the table to all_tables
                    except:
                        df['PDF'] = spec
                        df['Page_Number'] = page_number
                        er_dfs.append(df)

                elif df.shape[1] > 4 and page_number == 1:
                    df = df.T  # Transpose the DataFrame
                    df.loc[df[0].isin(fraction_lst), 0] = df.loc[df[0].isin(fraction_lst), 0].apply(convert_fraction_to_decimal)
                    df.columns = df.iloc[0]
                    df.drop(0, inplace=True)
                    mapping_table.append(df)

        # Combine all extracted tables into one DataFrame
        df_ = pd.concat(all_tables, ignore_index=True)
        df_['FROM'] = round(df_['FROM'].astype(float), 1)
        df_['TO'] = round(df_['TO'].astype(float), 1)

        # Process mapping table for NPS/DN values
        try:
            df_specs = pd.concat([mapping_table[1], mapping_table[2]], ignore_index=True)
        except:
            try:
                df_specs = mapping_table[1]
            except:
                df_specs = mapping_table[0]

        try:
            df_specs['NPS'] = round(df_specs['NPS'].astype(float), 1)
        except:
            df_specs['DN'] = round(df_specs['DN'].astype(float), 1)

        result_df = pd.DataFrame()  # DataFrame to store the final results

        # Iterate over each row in the main DataFrame (df_)
        for index, row in df_.iterrows():
            try:
                # Filter sizes from df_specs that fall within the range [FROM, TO]
                filtered_sizes = df_specs[(df_specs['NPS'] >= row['FROM']) & (df_specs['NPS'] <= row['TO'])]
            except:
                filtered_sizes = df_specs[(df_specs['DN'] >= row['FROM']) & (df_specs['DN'] <= row['TO'])]

            # Extract rating from the DESCRIPTION using regex pattern
            matches = re.findall(pattern, row['DESCRIPTION'])
            rating = matches[0] if matches else '-'

            # Determine the SCH1 value (either from 'SCHEDULE' or 'THICKNESS')
            for _, rw in filtered_sizes.iterrows():
                sch1 = rw['SCHEDULE'] if rw['SCHEDULE'] != '-' else rw['THICKNESS']
                nips = rw.get('NPS', rw.get('DN', '-'))  # Use NPS or DN based on available column

                # Append the data for the row to result_df
                result_df = result_df.append({
                    'ITEM_GROUP': row['ITEM_GROUP'],
                    'ITEM': row['ITEM'],
                    'FROM': row['FROM'],
                    'TO': row['TO'],
                    'SIZE1 (NPS)': nips,
                    'SIZE2 (NPS)': "",
                    'DESCRIPTION': row['DESCRIPTION'],
                    'MATERIAL': row['MATERIAL'],
                    'ENDS': row['ENDS'],
                    'STANDARD': row['STANDARD'],
                    'RATING': rating,
                    'SCH1': sch1,
                    'PIPING SPEC': spec
                }, ignore_index=True)

    return result_df, er_dfs  # Return the final result and any error DataFrames

# Example usage
pdf_file = "C:/Users/ahmad.makki/Desktop/AGES/AGES/Section E and F Pipe Class Index and Detailed piping Class/Section F Detailed Pipe class/AC1A1B-FA Rev1.pdf"  # Replace with your PDF file path
result_df, er_dfs = extract_table(pdf_file)

# Display or save the result
print(result_df)